# Block Continuity CNN Training

Train 1D CNN model for block continuity detection.

**Workflow:**
1. Mount Google Drive
2. Load CSV dataset (64-dim features)
3. Train CNN model
4. Export to ONNX
5. Download model

**Requirements:**
- Dataset CSV in Google Drive: `ML/datasets/continuity_dataset.csv`
- GPU runtime recommended (Runtime > Change runtime type > T4 GPU)

## 1. Setup Environment

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install dependencies
!pip install onnx onnxruntime tensorboard -q

import sys
print(f"Python version: {sys.version}")

## 2. Define CNN Model

In [ ]:
import torch
import torch.nn as nn
from typing import List

class ContinuityCNN1D(nn.Module):
    """
    1D CNN for block continuity classification

    Input: (batch, 64) feature vector
    Output: (batch, 2) logits

    Architecture:
    - Reshape: (batch, 64) -> (batch, 1, 64)
    - Conv1d layers: 32 -> 64 -> 128 channels
    - Global Average Pooling
    - FC classifier: 128 -> 64 -> 2
    """

    def __init__(
        self,
        input_dim: int = 64,
        num_classes: int = 2,
        channels: List[int] = None,
        kernel_sizes: List[int] = None,
        dropout: float = 0.3,
        use_batch_norm: bool = True
    ):
        super().__init__()

        if channels is None:
            channels = [32, 64, 128]
        if kernel_sizes is None:
            kernel_sizes = [5, 3, 3]

        self.input_dim = input_dim
        self.num_classes = num_classes

        # Build conv layers
        conv_layers = []
        in_channels = 1

        for out_channels, kernel_size in zip(channels, kernel_sizes):
            padding = kernel_size // 2
            conv_layers.append(
                nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding)
            )
            if use_batch_norm:
                conv_layers.append(nn.BatchNorm1d(out_channels))
            conv_layers.append(nn.ReLU(inplace=True))
            conv_layers.append(nn.Dropout(dropout))
            in_channels = out_channels

        self.conv_layers = nn.Sequential(*conv_layers)
        self.gap = nn.AdaptiveAvgPool1d(1)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(channels[-1], 64),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(64, num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.unsqueeze(1)  # (batch, 1, 64)
        x = self.conv_layers(x)
        x = self.gap(x)
        x = self.classifier(x)
        return x


# Test model
model = ContinuityCNN1D()
test_input = torch.randn(32, 64)
output = model(test_input)
print(f"Model created successfully!")
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print(f"Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## 3. Load Dataset

In [ ]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Configuration
DATASET_PATH = "/content/drive/MyDrive/ML/datasets/continuity_dataset.csv"
OUTPUT_DIR = "/content/drive/MyDrive/ML/models/continuity"

# Create output directory
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)


class ContinuityDataset(Dataset):
    """Dataset for continuity features"""

    def __init__(self, features: np.ndarray, labels: np.ndarray):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


def load_dataset(csv_path: str, test_size: float = 0.2):
    """Load and split dataset"""
    print(f"Loading dataset from {csv_path}...")
    df = pd.read_csv(csv_path)
    print(f"Total samples: {len(df)}")

    # Extract features (columns f0-f63)
    feature_cols = [f"f{i}" for i in range(64)]
    X = df[feature_cols].values.astype(np.float32)

    # Extract labels
    y = df["is_continuous"].values.astype(np.int64)

    # Print class distribution
    unique, counts = np.unique(y, return_counts=True)
    print(f"Class distribution:")
    for label, count in zip(unique, counts):
        print(f"  Class {label}: {count} ({count/len(y)*100:.1f}%)")

    # Split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )

    print(f"Train samples: {len(X_train)}")
    print(f"Val samples: {len(X_val)}")

    # Normalize features
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0) + 1e-8
    X_train = (X_train - mean) / std
    X_val = (X_val - mean) / std

    # Save normalization params
    norm_params = {"mean": mean.tolist(), "std": std.tolist()}

    return (
        ContinuityDataset(X_train, y_train),
        ContinuityDataset(X_val, y_val),
        norm_params
    )


# Load dataset
train_dataset, val_dataset, norm_params = load_dataset(DATASET_PATH)

# Create data loaders
BATCH_SIZE = 256
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"\nBatch size: {BATCH_SIZE}")
print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

## 4. Training Configuration

In [ ]:
# Training hyperparameters
config = {
    "epochs": 100,
    "learning_rate": 1e-3,
    "weight_decay": 1e-4,
    "patience": 15,  # Early stopping patience
    "use_amp": True,  # Mixed precision training
}

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ContinuityCNN1D().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config["learning_rate"],
    weight_decay=config["weight_decay"]
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=config["epochs"]
)

# AMP scaler
scaler = torch.amp.GradScaler() if config["use_amp"] else None

print(f"Device: {device}")
print(f"Config: {config}")

## 5. Train Model

In [ ]:
from tqdm.auto import tqdm
import json

def train_epoch(model, loader, criterion, optimizer, scaler, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for features, labels in loader:
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()

        if scaler:
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(features)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * features.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    return total_loss / total, correct / total


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    for features, labels in loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        loss = criterion(outputs, labels)

        total_loss += loss.item() * features.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    return total_loss / total, correct / total


# Training loop
best_val_acc = 0
patience_counter = 0
history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

print("Starting training...")
print("=" * 60)

for epoch in range(config["epochs"]):
    # Train
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, scaler, device
    )

    # Evaluate
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    # Update scheduler
    scheduler.step()

    # Record history
    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    # Print progress
    lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1:3d}/{config['epochs']} | "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} | "
          f"LR: {lr:.2e}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0

        checkpoint = {
            "epoch": epoch + 1,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "val_acc": val_acc,
            "val_loss": val_loss,
            "config": config,
            "norm_params": norm_params
        }
        torch.save(checkpoint, f"{OUTPUT_DIR}/best_cnn.pt")
        print(f"  >> Saved best model (val_acc: {val_acc:.4f})")
    else:
        patience_counter += 1

    # Early stopping
    if patience_counter >= config["patience"]:
        print(f"\nEarly stopping at epoch {epoch+1}")
        break

print("=" * 60)
print(f"Training complete! Best val accuracy: {best_val_acc:.4f}")

## 6. Visualize Training

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss plot
axes[0].plot(history["train_loss"], label="Train")
axes[0].plot(history["val_loss"], label="Validation")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("Training and Validation Loss")
axes[0].legend()
axes[0].grid(True)

# Accuracy plot
axes[1].plot(history["train_acc"], label="Train")
axes[1].plot(history["val_acc"], label="Validation")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Training and Validation Accuracy")
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/training_curves.png", dpi=150)
plt.show()

print(f"\nTraining curves saved to {OUTPUT_DIR}/training_curves.png")

## 7. Export to ONNX

In [ ]:
import onnx

# Load best checkpoint
checkpoint = torch.load(f"{OUTPUT_DIR}/best_cnn.pt", weights_only=False)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

print(f"Loaded checkpoint from epoch {checkpoint['epoch']}")
print(f"Validation accuracy: {checkpoint['val_acc']:.4f}")

# Export to ONNX
dummy_input = torch.randn(1, 64).to(device)
onnx_path = f"{OUTPUT_DIR}/continuity_cnn.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=17,
    do_constant_folding=True,
    input_names=['features'],
    output_names=['logits'],
    dynamic_axes={
        'features': {0: 'batch_size'},
        'logits': {0: 'batch_size'}
    }
)

# Verify ONNX model
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

# Save metadata
metadata = {
    "input_dim": 64,
    "num_classes": 2,
    "model_type": "cnn1d",
    "norm_params": checkpoint["norm_params"],
    "val_accuracy": checkpoint["val_acc"],
    "training_epochs": checkpoint["epoch"],
    "label_map": {"0": "discontinuous", "1": "continuous"}
}

with open(f"{OUTPUT_DIR}/continuity_cnn.json", "w") as f:
    json.dump(metadata, f, indent=2)

# Get file sizes
onnx_size = os.path.getsize(onnx_path)
print(f"\nONNX model exported: {onnx_path}")
print(f"Model size: {onnx_size / 1024:.1f} KB")
print(f"Metadata saved: {OUTPUT_DIR}/continuity_cnn.json")

## 8. Verify ONNX Inference

In [ ]:
import onnxruntime as ort
import numpy as np

# Create ONNX Runtime session
session = ort.InferenceSession(onnx_path)

# Test inference
test_features = np.random.randn(10, 64).astype(np.float32)
ort_outputs = session.run(None, {'features': test_features})

print(f"ONNX Runtime inference test:")
print(f"  Input shape: {test_features.shape}")
print(f"  Output shape: {ort_outputs[0].shape}")

# Compare with PyTorch output
model.cpu()
with torch.no_grad():
    pt_output = model(torch.from_numpy(test_features)).numpy()

max_diff = np.abs(ort_outputs[0] - pt_output).max()
print(f"  Max difference from PyTorch: {max_diff:.2e}")

if max_diff < 1e-5:
    print("  ONNX export verified successfully!")
else:
    print("  Warning: Larger than expected difference")

## 9. Download Models

The trained models are saved to Google Drive at:
- `ML/models/continuity/continuity_cnn.onnx` - ONNX model for C++ inference
- `ML/models/continuity/continuity_cnn.json` - Metadata with normalization params
- `ML/models/continuity/best_cnn.pt` - PyTorch checkpoint

You can also download directly:

In [ ]:
from google.colab import files

print("Output files:")
for f in os.listdir(OUTPUT_DIR):
    path = os.path.join(OUTPUT_DIR, f)
    size = os.path.getsize(path)
    print(f"  {f}: {size/1024:.1f} KB")

# Download ONNX model (uncomment to download)
# files.download(f"{OUTPUT_DIR}/continuity_cnn.onnx")
# files.download(f"{OUTPUT_DIR}/continuity_cnn.json")

## Usage in C++

After downloading, copy files to your C++ project:

```bash
# Copy to Filerestore_CLI models directory
copy continuity_cnn.onnx Filerestore_CLI\x64\Release\models\continuity\
copy continuity_cnn.json Filerestore_CLI\x64\Release\models\continuity\
```

The `BlockContinuityDetector` class will automatically load the new model.